# Deep Learning for Tuberculosis Detection and Classification: A YOLO-v8 + Convolutional Block Attention Module Approach

By: John Abuel

### Overview of the TBX11K Dataset

Source;
Tuberculosis X-ray dataset contains 11200 X-ray images with corresponding bounding box annotations for tuberculosis (TB) areas, while the existing largest public TB datasets have much fewer X-ray images with corresponding image-level annotations. All images are with a size of 512x512. There are five main categories in this dataset: healthy, sick_but_non-tb, active_tb, latent_tb, and uncertain_tb. Also, the dataset includes the active_tb&latent_tb categorie. The authors split this dataset into train, val, and test sets, consisting of 6600, 1800, and 2800(3302) X-ray images, respectively. The proposed dataset enables the training of sophisticated detectors for high-quality computer-aided tuberculosis diagnosis (CTD). Authors reform the existing object detectors to adapt them to simultaneous image classification and TB area detection. These reformed detectors are trained and evaluated on the proposed TBX11K dataset and serve as the baselines for future research.



As a serious infectious disease, tuberculosis (TB) is one of the major threats to human health worldwide, leading to millions of death every year. Although early diagnosis and treatment can greatly improve the chances of survival, it remains a major challenge, especially in developing countries. Computer-aided tuberculosis diagnosis (CTD) is a promising choice for TB diagnosis due to the great successes of deep learning.

## Importing necessary libraries


In [7]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import shutil
from tqdm import tqdm
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 6)

## Configuration for TBX11K preprocessing

In [8]:
class Config:
    """Configuration for TBX11K preprocessing"""
    
    # Raw data paths (download from TBX11K dataset)
    RAW_DATA_ROOT = '../dataset/raw_data/TBX11K'
    RAW_IMAGES = os.path.join(RAW_DATA_ROOT, 'imgs')
    RAW_ANNOTATIONS = os.path.join(RAW_DATA_ROOT, 'annotations')
    
    # Processed data paths
    PROCESSED_ROOT = '../dataset/data/TBX11K'
    TRAIN_IMAGES = os.path.join(PROCESSED_ROOT, 'train', 'images')
    TRAIN_LABELS = os.path.join(PROCESSED_ROOT, 'train', 'labels')
    VAL_IMAGES = os.path.join(PROCESSED_ROOT, 'val', 'images')
    VAL_LABELS = os.path.join(PROCESSED_ROOT, 'val', 'labels')
    TEST_IMAGES = os.path.join(PROCESSED_ROOT, 'test', 'images')
    TEST_LABELS = os.path.join(PROCESSED_ROOT, 'test', 'labels')
    
    # Classification data (for classifier training)
    CLASSIFICATION_ROOT = '../dataset/data/classification'
    
    # Split ratios
    TRAIN_RATIO = 0.7
    VAL_RATIO = 0.15
    TEST_RATIO = 0.15
    
    # Image processing
    TARGET_SIZE = (640, 640)  # For YOLO
    NORMALIZE = True
    
    # Quality control
    MIN_IMAGE_SIZE = (224, 224)
    MAX_IMAGE_SIZE = (4096, 4096)
    MIN_BOX_AREA = 100  # pixels
    
config = Config()  

# Create directories

for path in [config.TRAIN_IMAGES, config.TRAIN_LABELS,
             config.VAL_IMAGES, config.VAL_LABELS,
             config.TEST_IMAGES, config.TEST_LABELS]:
    os.makedirs(path, exist_ok=True)


print(f"✓ Configuration loaded")
print(f"  Raw data: {config.RAW_DATA_ROOT}")
print(f"  Output: {config.PROCESSED_ROOT}")


✓ Configuration loaded
  Raw data: ../dataset/raw_data/TBX11K
  Output: ../dataset/data/TBX11K


## Loading and Parsing TBX11K Annotations

In [18]:
def load_tbx11k_annotations(annotation_file):
    """
    Load TBX11K annotations from JSON format
    
    TBX11K format:
    {
        "images": [...],
        "annotations": [...],
        "categories": [...]
    }
    """
    print("\nLoading TBX11K annotations...")
    
    with open(annotation_file, 'r') as f:
        data = json.load(f)
    
    # Parse images
    images = {}
    for img in data['images']:
        images[img['id']] = {
            'file_name': img['file_name'],
            'width': img.get('width', 0),
            'height': img.get('height', 0),
            'sex': img.get('sex', 'Unknown'),  # Often 'M' or 'F'
            'age': img.get('age', None),       # Integer age
            'global_category': img.get('syms', 'Unknown'), # Sometimes stored as 'syms' or implied
            'annotations': []
        }
    
    # Parse categories
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Parse annotations
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id in images:
            images[img_id]['annotations'].append({
                'bbox': ann['bbox'],  # [x, y, width, height]
                'category_id': ann['category_id'],
                'category_name': categories.get(ann['category_id'], 'unknown'),
                'area': ann.get('area', 0)
            })

    for img_id, img_data in images.items():
        if len(img_data['annotations']) > 0:
            img_data['global_category'] = 'TB'
        elif img_data['global_category'] == 'Unknown':
             # Fallback if specific metadata isn't found
            img_data['global_category'] = 'Non-TB (Healthy or Sick)'
    
    print(f"Loaded {len(images)} images")
    print(f"Categories: {list(categories.values())}")
    
    return images, categories

base_dir = os.path.abspath(os.path.join(os.getcwd(), '..')) # Go up from notebooks/
annotation_path = os.path.join(base_dir, 'dataset', 'raw_data', 'TBX11K', 'annotations', 'json', 'TBX11K_train.json')

if os.path.exists(annotation_path):
    images_data, categories = load_tbx11k_annotations(annotation_path)
    # Now run your EDA function
else:
    print(f"⚠ Annotation file not found at: {annotation_path}")


Loading TBX11K annotations...
Loaded 6600 images
Categories: ['ActiveTuberculosis', 'ObsoletePulmonaryTuberculosis', 'PulmonaryTuberculosis']


## Exploratory Data Analysis of TBX11K Dataset

In [ ]:
def perform_eda(images_data, config):
    """Comprehensive EDA including Pixel Intensity & Class-Specific Box Stats"""

    stats = {
        'num_images': len(images_data),
        'num_with_annotations': 0,
        'num_without_annotations': 0,
        'total_boxes': 0,
        'boxes_per_image': [],
        # Box dimensions
        'box_areas_active': [],
        'box_areas_latent': [],
        'box_areas_all': [],
        # Pixel Intensity Stats
        'lesion_intensity': [],
        'background_intensity': [],
        'contrast_ratios': [],
        # Class Counts
        'category_counts': defaultdict(int)
    }
    
    print("Analyzing images and pixel intensities...")
    
    for img_id, img_info in tqdm(images_data.items(), desc="Processing"):
        img_path = os.path.join(config.RAW_IMAGES, img_info['file_name'])
        
        # Check file existence
        if not os.path.exists(img_path):
            continue
            
        annotations = img_info['annotations']
        num_boxes = len(annotations)
        stats['boxes_per_image'].append(num_boxes)
        
        # --- 1. Class & Box Analysis ---
        if num_boxes > 0:
            stats['num_with_annotations'] += 1
            stats['total_boxes'] += num_boxes
            
            # Load image for pixel analysis (Grayscale is sufficient for intensity)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            mask = None
            if img is not None:
                mask = np.zeros_like(img)

            for ann in annotations:
                cat_name = ann['category_name']
                stats['category_counts'][cat_name] += 1
                
                # Box Geometry
                x, y, w, h = map(int, ann['bbox'])
                area = w * h
                stats['box_areas_all'].append(area)
                
                # Class-specific areas (checking for partial string matches for safety)
                if 'Active' in cat_name:
                    stats['box_areas_active'].append(area)
                elif 'Obsolete' in cat_name or 'Latent' in cat_name:
                    stats['box_areas_latent'].append(area)
                
                # Prepare Mask
                if img is not None:
                    # Draw filled rectangle for the lesion
                    cv2.rectangle(mask, (x, y), (x+w, y+h), 255, -1)
            
            # --- 2. Pixel Intensity Analysis ---
            if img is not None:
                # Pixels inside boxes (Lesions)
                lesion_pix = img[mask == 255]
                # Pixels outside boxes (Background/Healthy Lung)
                bg_pix = img[mask == 0]
                
                if len(lesion_pix) > 0 and len(bg_pix) > 0:
                    l_mean = np.mean(lesion_pix)
                    b_mean = np.mean(bg_pix)
                    stats['lesion_intensity'].append(l_mean)
                    stats['background_intensity'].append(b_mean)
                    # Avoid divide by zero
                    stats['contrast_ratios'].append(l_mean / (b_mean + 1e-6))
        else:
            stats['num_without_annotations'] += 1

    # --- REPORTING ---
    print("\n" + "="*50)
    print("      DATASET ANALYSIS REPORT      ")
    print("="*50)

    # 1. Global Class Distribution
    print(f"\n IMAGE-LEVEL DISTRIBUTION")
    print(f"Total Images: {stats['num_images']}")
    print(f"  - With TB (Annotations): {stats['num_with_annotations']} ({stats['num_with_annotations']/stats['num_images']*100:.1f}%)")
    print(f"  - No TB (Healthy/Sick):  {stats['num_without_annotations']} ({stats['num_without_annotations']/stats['num_images']*100:.1f}%)")

    # 2. Annotation Class Distribution
    print(f"\n ANNOTATION CLASS DISTRIBUTION (Active vs Latent)")
    total_boxes = stats['total_boxes']
    print(f"Total Boxes: {total_boxes}")
    for cat, count in sorted(stats['category_counts'].items()):
        ratio = (count / total_boxes) * 100 if total_boxes > 0 else 0
        print(f"  - {cat:<30}: {count} ({ratio:.1f}%)")


    return stats


# Perform EDA
if images_data:
    dataset_stats = perform_eda(images_data, config)

Analyzing images and pixel intensities...


Processing: 100%|██████████| 6600/6600 [00:05<00:00, 1190.38it/s] 


      DATASET ANALYSIS REPORT      

📸 IMAGE-LEVEL DISTRIBUTION
Total Images: 6600
  - With TB (Annotations): 599 (9.1%)
  - No TB (Healthy/Sick):  6001 (90.9%)

🏷️ ANNOTATION CLASS DISTRIBUTION (Active vs Latent)
Total Boxes: 902
  - ActiveTuberculosis            : 724 (80.3%)
  - ObsoletePulmonaryTuberculosis : 178 (19.7%)

📦 BOUNDING BOX GEOMETRY
  - Overall Mean Area:      16151 px²
  - Active TB Mean Area:    16870 px²
  - Latent TB Mean Area:    13224 px²
    (Latent boxes are 21.6% smaller on average)

💡 PIXEL INTENSITY & CONTRAST (CBAM Justification)
  - Avg Lesion Intensity:     121.2 (0-255)
  - Avg Background Intensity: 132.7 (0-255)
  - Mean Contrast Ratio:      0.920

  ✅ CONCLUSION: Low contrast detected (< 1.15).
     Lesions are visually subtle compared to the background.
     This strictly justifies using CBAM attention to boost feature extraction.
